# Tune Hyperparameters

## 1. Connect to workspace

In [ ]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

## 2. Prepare a training script

### Hyperdrive Training Folder Set Up

In [ ]:
import os, shutil

# Create a folder for the training script
experiment_folder = 'training-hyperdrive'
os.makedirs(experiment_folder, exist_ok=True)

shutil.copy(
    '../1. Analysis and Preprocessing/preprocessed_datasets/market_segmentation.csv',
    os.path.join(experiment_folder, "market_segmentation.csv")
)
shutil.copy(
    '../1. Analysis and Preprocessing/preprocessed_datasets/market_segmentation_interaction.csv',
    os.path.join(experiment_folder, "market_segmentation_interaction.csv")
)

print('Folder ready.')

### Logistic Regression

In [ ]:
%%writefile $experiment_folder/lg-training.py

# Import libraries
import argparse, joblib, os
from azureml.core import Run
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.preprocessing import LabelBinarizer

# Get the experiment run context
run = Run.get_context()

# Get script arguments
parser = argparse.ArgumentParser()

# Set regularization and solver hyperparameters
parser = argparse.ArgumentParser()
parser.add_argument('--reg_rate', type=float, dest='reg', default=0.01)
parser.add_argument('--solver', type=str, dest='solver', default='lbfgs')
args = parser.parse_args()
reg = args.reg
solver = args.solver

# Log Hyperparameter values
run.log('reg',  np.float(args.reg))
run.log('solver', args.solver)

# load the market segmentation dataset
print("Loading Data...")
market_segmentation = pd.read_csv('market_segmentation_interaction.csv')

# Separate features and labels
X, y = market_segmentation.drop(columns="Segmentation"), market_segmentation.Segmentation

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Train a logistic regression model
print('Training a logistic regression model with regularization rate of', reg)
model = LogisticRegression(C=1/reg, solver=solver).fit(X_train, y_train)

# Calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# Calculate AUC
label_binarizer = LabelBinarizer().fit(y_train)
y_onehot_test = label_binarizer.transform(y_test)
y_scores = model.predict_proba(X_test)
for class_of_interest in ["A", "B", "C", "D"]:
    class_id = np.flatnonzero(label_binarizer.classes_ == class_of_interest)[0]
    auc = roc_auc_score(y_onehot_test[:,class_id],y_scores[:,class_id])
    print(f'AUC {class_of_interest} vs rest: ' + str(auc))
    run.log(f'AUC {class_of_interest} vs rest', np.float(auc))

# Save the model in the run outputs
os.makedirs('outputs', exist_ok=True)
joblib.dump(value=model, filename='outputs/lg-model.pkl')

run.complete()

### Random Forest

In [ ]:
%%writefile $experiment_folder/rf-training.py

# Import libraries
import argparse, joblib, os
from azureml.core import Run
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.preprocessing import LabelBinarizer

# Get the experiment run context
run = Run.get_context()

# Get script arguments
parser = argparse.ArgumentParser()

# Set hyperparameters
parser = argparse.ArgumentParser()
parser.add_argument('--max_features', type=int, dest='max_features', default=4)
parser.add_argument('--max_depth', type=int, dest='max_depth', default=None)
parser.add_argument('--min_samples_split', type=int, dest='min_samples_split', default=2)
parser.add_argument('--min_samples_leaf', type=int, dest='min_samples_leaf', default=1)
args = parser.parse_args()
max_features = args.max_features
max_depth = args.max_depth
min_samples_split = args.min_samples_split
min_samples_leaf = args.min_samples_leaf

# Log Hyperparameter values
run.log('Maximum depth of tree', max_depth)
run.log('Maximum features per split', max_features)
run.log('Minimum samples requierd for split', min_samples_split)
run.log('Minimum samples per leaf', min_samples_leaf)

# load the market segmentation dataset
print("Loading Data...")
market_segmentation = pd.read_csv('market_segmentation.csv')

# Separate features and labels
X, y = market_segmentation.drop(columns="Segmentation"), market_segmentation.Segmentation

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Train a logistic regression model
print(
    'Training a random forest model with max features of', max_features,
    'max depth of', max_depth,
    'min samples split of', min_samples_split,
    'and min_samples_leaf of', min_samples_leaf
)
model = RandomForestClassifier(
    max_features=max_features,
    max_depth=max_depth,
    min_samples_split=min_samples_split,
    min_samples_leaf=min_samples_leaf
).fit(X_train, y_train)

# Calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# Calculate AUC
label_binarizer = LabelBinarizer().fit(y_train)
y_onehot_test = label_binarizer.transform(y_test)
y_scores = model.predict_proba(X_test)
for class_of_interest in ["A", "B", "C", "D"]:
    class_id = np.flatnonzero(label_binarizer.classes_ == class_of_interest)[0]
    auc = roc_auc_score(y_onehot_test[:,class_id],y_scores[:,class_id])
    print(f'AUC {class_of_interest} vs rest: ' + str(auc))
    run.log(f'AUC {class_of_interest} vs rest', np.float(auc))

# Save the model in the run outputs
os.makedirs('outputs', exist_ok=True)
joblib.dump(value=model, filename='outputs/rf-model.pkl')

run.complete()

## 3. Create compute

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "aguo921"

try:
    # Check for existing compute target
    training_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If it doesn't already exist, create it
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=2)
        training_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        training_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)
    

## 4. Create environment

In [ ]:
%%writefile $experiment_folder/hyperdrive_env.yml
name: batch_environment
dependencies:
- python=3.6.2
- scikit-learn
- pandas
- numpy
- pip
- pip:
  - azureml-defaults

In [ ]:
from azureml.core import Experiment, ScriptRunConfig, Environment
from azureml.train.hyperdrive import GridParameterSampling, HyperDriveConfig, PrimaryMetricGoal, choice

# Create a Python environment for the experiment
hyper_env = Environment.from_conda_specification("experiment_env", experiment_folder + "/hyperdrive_env.yml")

## 5. Run a hyperparameter tuning experiment

### Logistic Regression

In [ ]:
# Create a script config
script_config = ScriptRunConfig(
    source_directory=experiment_folder,
    script='lg-training.py',
    environment=hyper_env,
    compute_target = training_cluster
)

# Sample a range of parameter values
params = GridParameterSampling(
    {
        '--reg': choice(0.3, 0.4, 0.5, 0.6),
        '--solver': choice('newton-cg', 'lbfgs')
    }
)

# Configure hyperdrive settings
hyperdrive = HyperDriveConfig(
    run_config=script_config, 
    hyperparameter_sampling=params, 
    policy=None, # No early stopping policy
    primary_metric_name='Accuracy', # Find the highest Accuracy metric
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
    max_total_runs=12, # Restict the experiment to 16 iterations
    max_concurrent_runs=2 # Run up to 2 iterations in parallel
) 

# Run the experiment
experiment = Experiment(workspace=ws, name='segmentation-hyperdrive-lg')
run_lg = experiment.submit(config=hyperdrive)

# Show the status in the notebook as the experiment runs
run_lg.wait_for_completion()

### Random Forest

In [ ]:
# Create a script config
script_config = ScriptRunConfig(
    source_directory=experiment_folder,
    script='rf-training.py',
    environment=hyper_env,
    compute_target = training_cluster
)

# Sample a range of parameter values
params = GridParameterSampling(
    {
        '--max_features': choice(3),
        '--max_depth': choice(12, 15),
        '--min_samples_split': choice(10, 11),
        '--min_samples_leaf': choice(11, 12)
    }
)

# Configure hyperdrive settings
hyperdrive = HyperDriveConfig(
    run_config=script_config, 
    hyperparameter_sampling=params, 
    policy=None, # No early stopping policy
    primary_metric_name='Accuracy', # Find the highest Accuracy metric
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
    max_total_runs=16, # Restict the experiment to 10 iterations
    max_concurrent_runs=2 # Run up to 2 iterations in parallel
) 

# Run the experiment
experiment = Experiment(workspace=ws, name='segmentation-hyperdrive-rf')
run_rf = experiment.submit(config=hyperdrive)

# Show the status in the notebook as the experiment runs
run_rf.wait_for_completion()

## 5. Register the best performing model

In [ ]:
from azureml.core import Model

### Logistic Regression

In [ ]:
# Print all child runs, sorted by the primary metric
for child_run in run_lg.get_children_sorted_by_primary_metric():
    print(child_run)

# Get the best run, and its metrics and arguments
best_run = run_lg.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
script_arguments = best_run.get_details() ['runDefinition']['arguments']
print('Best Run Id: ', best_run.id)
print(' -AUC A vs rest:', best_run_metrics['AUC A vs rest'])
print(' -AUC B vs rest:', best_run_metrics['AUC B vs rest'])
print(' -AUC C vs rest:', best_run_metrics['AUC C vs rest'])
print(' -AUC D vs rest:', best_run_metrics['AUC D vs rest'])
print(' -Accuracy:', best_run_metrics['Accuracy'])
print(' -Arguments:',script_arguments)

In [ ]:
# Register model
best_run.register_model(
    model_path='outputs/lg-model.pkl', model_name='segmentation-lg',
    tags={'Training context':'Hyperdrive'},
    properties={
        'AUC A vs rest': best_run_metrics['AUC A vs rest'],
        'AUC B vs rest': best_run_metrics['AUC B vs rest'],
        'AUC C vs rest': best_run_metrics['AUC C vs rest'],
        'AUC D vs rest': best_run_metrics['AUC D vs rest'],
        'Accuracy': best_run_metrics['Accuracy']
    }
)

### Random Forest

In [ ]:
# Print all child runs, sorted by the primary metric
for child_run in run_rf.get_children_sorted_by_primary_metric():
    print(child_run)

# Get the best run, and its metrics and arguments
best_run = run_rf.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
script_arguments = best_run.get_details() ['runDefinition']['arguments']
print('Best Run Id: ', best_run.id)
print(' -AUC A vs rest:', best_run_metrics['AUC A vs rest'])
print(' -AUC B vs rest:', best_run_metrics['AUC B vs rest'])
print(' -AUC C vs rest:', best_run_metrics['AUC C vs rest'])
print(' -AUC D vs rest:', best_run_metrics['AUC D vs rest'])
print(' -Accuracy:', best_run_metrics['Accuracy'])
print(' -Arguments:',script_arguments)

In [ ]:
# Register model
best_run.register_model(
    model_path='outputs/rf-model.pkl', model_name='segmentation-rf',
    tags={'Training context':'Hyperdrive'},
    properties={
        'AUC A vs rest': best_run_metrics['AUC A vs rest'],
        'AUC B vs rest': best_run_metrics['AUC B vs rest'],
        'AUC C vs rest': best_run_metrics['AUC C vs rest'],
        'AUC D vs rest': best_run_metrics['AUC D vs rest'],
        'Accuracy': best_run_metrics['Accuracy']
    }
)

### All models

In [ ]:
# List registered models
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')